In [1]:
import folium 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
%matplotlib inline

In [2]:
df = pd.read_csv('toronto_latlng_df.csv')

In [9]:
df.drop(df.columns[0], axis=1, inplace=True)

In [11]:
df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785665,-79.158725
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
...,...,...,...,...,...
98,M9N,York,Weston,43.704845,-79.517546
99,M9P,Etobicoke,Westmount,43.696505,-79.530252
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.686810,-79.557284
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.743145,-79.584664


In [13]:
df['Borough'].unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       'Mississauga', 'Etobicoke'], dtype=object)

In [26]:
from folium.plugins import MarkerCluster

toronto_coord = (43.6532, -79.3832)

map = folium.Map(toronto_coord, zoom_start=12)
cluster = MarkerCluster()

for i, row in df.iterrows():
    
    label = '{} {}'.format(row['Postcode'], row['Neighbourhood'])
    marker = folium.Marker(
        location=(row['Latitude'], row['Longitude']),
        popup=label        
    )
    cluster.add_child(marker)

map.add_child(cluster)